In [1]:
%load_ext autoreload
%autoreload 2


from util2 import *
from myalgorithm_0629_1 import algorithm_test
from myalgorithm_0629_2 import algorithm

- 실험해볼 내용들

1. 서로 비슷한 번들이나 주문끼리 병합 시도하는게 시간 효율적이지 않나?
2. 서로 비슷하다의 기준은 어떻게 잡으면 좋을까?
3. 초반에 어느 정도 최적화된 값에서 시작하면 어떨까?

## 초기에 비슷한 번들로 먼저 묶기

In [14]:
## Basic Code
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

## Custom Code
def get_init_bundle(weight1, weight2):
    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    car_rider = ALL_RIDERS[2]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            dist1 = DIST[i][j]
            dist2 = DIST[i + K][j + K]

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(K))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue
        
        new_bundle = try_merging_bundles4(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]

    return result_bundles, sum((bundle.cost for bundle in result_bundles)) / K

min_init_cost = 100000000000000000
min_init_cost_bundle = []
for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundle, cost = get_init_bundle(weight1, weight2)

        print(weight1, weight2, cost)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundle = bundle

# print(min_init_cost_bundle)
print(min_init_cost)

0 -3 3887.276
0 -2 3895.8479999999995
0 -1.5 3845.8639999999996
0 -1 3899.5299999999993
0 -0.5 3912.446
0 0 3977.244
0 0.5 3870.0180000000005
1 -3 3800.5499999999993
1 -2 3749.6819999999993
1 -1.5 3672.6479999999997
1 -1 3772.486
1 -0.5 3864.4339999999997
1 0 3874.47
1 0.5 3883.886
3672.6479999999997


## myalgorithm 모듈로 이용하는 방식으로 제작 및 버그 확인

In [27]:
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'

timelimit = 60

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])

algorithm_test(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

0 -3 3887.276
0 -2 3895.8479999999995
0 -1.5 3845.8639999999996
0 -1 3899.5299999999993
0 -0.5 3912.446
0 0 3977.244
0 0.5 3870.0180000000005
1 -3 3800.5499999999993
1 -2 3749.6819999999993
1 -1.5 3672.6479999999997
1 -1 3772.486
1 -0.5 3864.4339999999997
1 0 3874.47
1 0.5 3883.886
[Bundle(all_orders, CAR, [3, 0, 16], [3, 0, 16], 96, True), Bundle(all_orders, CAR, [1, 24], [24, 1], 52, True), Bundle(all_orders, BIKE, [32, 2, 38], [2, 32, 38], 94, True), Bundle(all_orders, CAR, [7, 4], [7, 4], 43, True), Bundle(all_orders, BIKE, [5, 25, 68], [5, 68, 25], 70, True), Bundle(all_orders, BIKE, [6, 33, 28], [6, 33, 28], 80, True), Bundle(all_orders, BIKE, [8, 9], [8, 9], 54, True), Bundle(all_orders, BIKE, [10, 40, 26], [26, 10, 40], 78, True), Bundle(all_orders, BIKE, [11, 12, 43], [43, 11, 12], 66, True), Bundle(all_orders, BIKE, [13, 34, 18], [34, 18, 13], 76, True), Bundle(all_orders, BIKE, [29, 45, 14], [14, 45, 29], 84, True), Bundle(all_orders, CAR, [17, 15], [17, 15], 36, True), Bund

> 기존에는 출발지 사이의 거리와 도착지 사이의 거리만 이용하였으며, 추가적으로 주문 완료 시간과 주문 데드라인 시간, 출발지와 도착지 사이의 거리를 각각 가중치를 주어봤다. 그 결과 확실히 기존에 비해 총 비용이 줄어드는 경우가 있으나, 그렇게 크게 줄어든다는 생각은 안들며, 테스트케이스마다 최적의 가중치 조합이 다르다. 그러므로 최적의 가중치 조합을 시간 부담이 되지 않는 선에서 돌려보는 것이 괜찮을 것 같다.

## BASSLINE의 라이더 바꾸는 코드를 이용해서 최적화 테스트

### 파일 하나 확인

In [6]:
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'

timelimit = 60

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])

solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

## 결과 실현 가능성 확인

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])

3649.296


### 파일 여러 개 확인

In [5]:
# problem_file = '../alg_test_problems_20240429/TEST_K200_2.json'

for stage in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{stage}.json'

    timelimit = 60

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])

    solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

    ## 결과 실현 가능성 확인

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    print(stage, checked_solution['avg_cost'])

1 4086.0519999999997
2 3771.685
3 3398.1214999999997
4 3040.9184999999998
5 2625.2606666666675
6 3296.4866666666676
7 3935.8279999999995
8 3775.07
9 3312.1894999999995
10 2909.1584999999995
11 2788.181333333334
12 3307.9760000000015
13 5245.33
14 4623.004
15 4454.261499999999
16 3758.289
17 3703.3740000000016
18 4084.696666666666


### 배달원을 바꾸는 기존 코드를 작동할 때 마감기한 제한이 어긋나는 문제 디버깅 (완료 후 윗 코드에 반영)

In [2]:
# problem_file = '../alg_test_problems_20240429/TEST_K200_2.json'
problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_4.json' # order이 할당이 안된 문제 발생 -> 사실 deadline 문제가 발생한 거였음

# problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{stage}.json'

timelimit = 60

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])

solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

# print(all_bundles)

#-----------------------------------------
# print(1234)

# print(sum((bundle.cost for bundle in all_bundles)) / K)

# for bundle in all_bundles:
#     new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
#     if new_rider is not None:
#         old_rider = bundle.rider

#         print(111)
#         print(test_route_feasibility(ALL_ORDERS, bundle.rider, bundle.shop_seq, bundle.dlv_seq))


#         print(bundle)

#         check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
#         if check_result:
#             bundle.shop_seq = check_result[0]
#             bundle.dlv_seq = check_result[1]
#             bundle.rider = check_result[2]
#             bundle.update_cost()

#             old_rider.available_number += 1
#             new_rider.available_number -= 1

#         print(222)
#         print(test_route_feasibility(ALL_ORDERS, bundle.rider, bundle.shop_seq, bundle.dlv_seq))

# solution = [
#         # rider type, shop_seq, dlv_seq
#         [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
#         for bundle in all_bundles
# ]

with open(problem_file, 'r') as f:
    prob = json.load(f)
K = prob['K']
ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]
DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

# print(sum((bundle.cost for bundle in all_bundles)) / K)
checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

> inplace하게 바꾸는 try_bundle_rider_changing 대신 함수 밖에서 바꿔주는 check_bundle_rider_changing으로 바꿨더니 주문 데드라인 넘게 되고 그것을 확인 못하는 문제가 해결되었음

### 제출용 (배달원 바꾸는 코드 디버깅은 실패)

In [91]:
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'

timelimit = 60

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])

solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

In [ ]:
with open(problem_file, 'r') as f:
    prob = json.load(f)
K = prob['K']
ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]
DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

# checked_solution